In [1]:
import math
import json
import pickle
import numpy as np
import falconn
import timeit
from __future__ import print_function
import psycopg2
import pandas as pd
from configparser import ConfigParser
import requests
import pyphi
import re

In [2]:
def call_embedding_ws(name):
    url = 'http://54.36.53.127:8009/embedding'
    headers = {"content-type": "application/json"}
    response = requests.post(url, json=[name])
    
    arr = pyphi.jsonify.loads(response.text)
    x = np.array(arr)
    x = x.astype(dtype=np.float32)
    return x[0]

def reverse_name(name):
    a=name.split()
    a.reverse()
    return " ".join(a)

def cos_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)
    return dot_product

def sort_by_distance(query_vector, result_vectors):
    for v in result_vectors:
        cos_similarity(query_vector, v)
        
        
def query_lhs_table_by_distance(query, lhs_table, distance, pairs, column, vectors):
    response = lhs_table.find_near_neighbors(query, distance)    
    return process_lhs_table_response(query, response, pairs, column, vectors)
    
def query_lhs_table_by_number(query, lhs_table, number, pairs, column, vectors):
    response = lhs_table.find_k_nearest_neighbors(query, k=number)    
    return process_lhs_table_response(query, response, pairs, column, vectors)

def query_lhs_table_nearest(query, lhs_table, pairs, column, vectors):
    response = lhs_table.find_nearest_neighbor(query)
    return process_lhs_table_response(query, [response], pairs, column, vectors)

def process_lhs_table_response(query, response, pairs, column, vectors):
    df = pd.DataFrame(index=range(len(response)), columns=['id', 'name', 'cosine'])   
    i = 0
    
    for resp in response:
        name = pairs.get_value(resp, column)
        df.set_value(index=i, col='id', value=resp)
        df.set_value(index=i, col='name', value=name)
        df.set_value(index=i, col='cosine', value=cos_similarity(query, vectors[resp]))
        i = i + 1
     
    df.sort_values('cosine', ascending=False)
    df = df.reset_index(drop=True)
    return df


def is_arabic(name):
    res = re.findall(r'[\u0600-\u06FF]+',name)
    if len(res) == 0:
        return False
    else:
        return True

In [4]:

pairs = pd.read_csv('data/test-sets/eng_eng/pos_eng_eng_1x1.tsv', sep='\t', names=['flag', 'left', 'right'])
pairs['left_vector'] = ''
pairs['right_vector'] = ''


i = 0
j = 0
ws_params = []


for index, row in pairs.iterrows():
    ws_params.append(row['left'])
    i = i + 1
    if i % 512 == 0:
        ws_result = call_embedding_ws(ws_params)   
        for vect in ws_result:
            v_str = np.array2string(vect, separator=',', max_line_width=2000).strip("[]").replace(" ", "")
            pairs['left_vector'][j] = v_str
            j = j + 1
        ws_params = []
        print(i)
        
if len(ws_params) > 0:
    ws_result = call_embedding_ws(ws_params)
    for vect in ws_result:
        v_str = np.array2string(vect, separator=',', max_line_width=2000).strip("[]").replace(" ", "")
        pairs['left_vector'][j] = v_str
        j = j + 1

i = 0
j = 0
ws_params = []
        
for index, row in pairs.iterrows():
    ws_params.append(row['right'])
    i = i + 1
    if i % 512 == 0:
        ws_result = call_embedding_ws(ws_params)   
        for vect in ws_result:
            v_str = np.array2string(vect, separator=',', max_line_width=2000).strip("[]").replace(" ", "")
            pairs['right_vector'][j] = v_str
            j = j + 1
        ws_params = []
        print(i)
        
if len(ws_params) > 0:
    ws_result = call_embedding_ws(ws_params)
    for vect in ws_result:
        v_str = np.array2string(vect, separator=',', max_line_width=2000).strip("[]").replace(" ", "")
        pairs['right_vector'][j] = v_str
        j = j + 1
        
print(pairs)
        
pairs.to_csv('data/test-sets/eng_eng/pos_eng_eng_1x1_vectors.tsv', sep='\t')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)